# Heath_AI_Near_Me_Map_Simulation_K_Foo_ 23th_Oct 2020

In [46]:
import math
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning
import minisom

In [47]:
dtf = pd.read_csv('stores_10_8_2020.csv')

In [48]:
filter = "Las Vegas"
dtf = dtf[dtf["City"]==filter][["City","Street Address","Longitude","Latitude"]].reset_index(drop=True)
dtf = dtf.reset_index().rename(columns={"index":"id"})
dtf.head()


,id,City,Street Address,Longitude,Latitude
0,0,Las Vegas,4507 Flamingo Rd,-115.20,36.12
1,1,Las Vegas,"475 E Windmill Lane, Fashion Show",-115.15,36.04
2,2,Las Vegas,"3200 LAS VEGAS BLVD. S., STE 1795",-115.17,36.13
3,3,Las Vegas,8350 W Cheyenne Ave,-115.28,36.22
4,4,Las Vegas,3730 LAS VEGAS BLVD S,-115.18,36.11


In [49]:
#Tells you the data above is how far Relative to 2255 E Centennial Parkway - 115.12,  36.28 in miles

Locations_less_than_5_miles=[]

for i in dtf['Longitude']:
    for x in dtf['Latitude']:
        radiusEarth= 3961 #miles. If you want km, put in 6373 km in instead
        lat1 = math.radians(36.14)  #3301 W. Sahara Ave.
        lon1 = math.radians(-115.19)  #3301 W. Sahara Ave.
        lat2 = math.radians(x)  #input latitude
        lon2 = math.radians(i)  #input longitude
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = round(radiusEarth * c,2) #in miles
        print(distance)


1.49
6.94
0.89
5.56
2.15
2.15
1.49
5.56
0.89
9.69
2.15
2.82
0.89
2.82
1.49
2.15
2.15
5.56
3.5
2.82
0.89
0.89
1.49
1.49
9.0
4.19
10.38
1.49
0.89
1.49
6.94
9.69
1.49
2.82
0.56
6.94
2.15
2.15
2.82
1.49
0.56
10.38
2.15
0.56
6.25
4.19
6.94
1.49
3.5
9.69
3.5
1.49
6.94
6.94
11.08
2.82
4.19
2.15
2.82
5.56
6.94
0.56
4.19
0.89
1.49
2.15
11.08
2.15
1.49
9.0
2.15
2.15
1.49
0.89
4.19
5.56
4.19
6.94
2.15
4.19
2.82
1.49
1.49
5.56
1.49
3.5
4.19
2.82
1.49
2.15
7.63
0.56
5.56
2.82
4.19
8.31
3.5
1.49
1.49
6.25
2.15
1.49
2.15
1.49
2.82
0.89
7.63
7.63
1.49
5.56
0.89
3.5
2.15
1.49
2.15
2.82
2.82
3.5
9.0
0.89
2.82
4.19
2.15
3.5
1.49
0.89
1.49
1.49
3.5
0.56
4.19
4.19
9.0
1.49
2.15
0.89
2.82
6.25
1.49
1.49
4.19
8.31
4.87
2.82
6.94
1.49
5.56
4.19
9.69
2.82
3.5
2.82
6.94
2.15
3.5
0.56
2.63
7.27
2.34
5.96
3.05
3.05
2.63
5.96
2.34
9.93
3.05
3.55
2.34
3.55
2.63
3.05
3.05
5.96
4.12
3.55
2.34
2.34
2.63
2.63
9.26
4.71
10.61
2.63
2.34
2.63
7.26
9.93
2.63
3.55
2.23
7.27
3.05
3.05
3.55
2.63
2.23
10.61
3.05
2.23
6.61
4.71

2.77
2.77
3.46
8.99
0.69
2.77
4.15
2.07
3.46
1.38
0.69
1.38
1.38
3.46
0.0
4.15
4.15
8.99
1.38
2.07
0.69
2.77
6.22
1.38
1.38
4.15
8.3
4.84
2.77
6.91
1.38
5.53
4.15
9.68
2.77
3.46
2.77
6.91
2.07
3.46
0.0
9.04
11.3
8.96
10.5
9.17
9.17
9.04
10.51
8.96
13.17
9.17
9.35
8.96
9.35
9.04
9.17
9.17
10.5
9.58
9.35
8.96
8.96
9.04
9.04
12.67
9.85
13.68
9.04
8.96
9.04
11.29
13.18
9.04
9.35
8.93
11.3
9.17
9.17
9.35
9.04
8.93
13.68
9.17
8.93
10.89
9.85
11.29
9.04
9.58
13.17
9.58
9.04
11.3
11.3
14.21
9.35
9.85
9.17
9.35
10.51
11.3
8.93
9.85
8.96
9.04
9.17
14.21
9.17
9.04
12.68
9.17
9.17
9.04
8.96
9.85
10.51
9.85
11.3
9.17
9.85
9.35
9.04
9.04
10.51
9.04
9.58
9.85
9.35
9.04
9.17
11.74
8.93
10.51
9.35
9.85
12.19
9.58
9.04
9.04
10.89
9.17
9.04
9.17
9.04
9.35
8.96
11.73
11.74
9.04
10.51
8.96
9.58
9.17
9.04
9.17
9.35
9.35
9.58
12.67
8.96
9.35
9.85
9.17
9.58
9.04
8.96
9.04
9.04
9.58
8.93
9.85
9.85
12.67
9.04
9.17
8.96
9.35
10.89
9.04
9.04
9.85
12.19
10.16
9.35
11.29
9.04
10.5
9.85
13.17
9.35
9.58
9.35
11.29
9.

2.63
2.23
10.61
3.05
2.23
6.61
4.71
7.26
2.63
4.12
9.93
4.12
2.63
7.27
7.27
11.28
3.55
4.71
3.05
3.55
5.96
7.27
2.23
4.71
2.34
2.63
3.05
11.28
3.05
2.63
9.26
3.05
3.05
2.63
2.34
4.71
5.96
4.71
7.27
3.05
4.71
3.55
2.63
2.63
5.96
2.63
4.11
4.71
3.55
2.63
3.05
7.93
2.23
5.96
3.55
4.71
8.59
4.11
2.63
2.63
6.61
3.05
2.63
3.05
2.63
3.55
2.34
7.93
7.93
2.63
5.96
2.34
4.11
3.05
2.63
3.05
3.55
3.55
4.12
9.26
2.34
3.55
4.71
3.05
4.12
2.63
2.34
2.63
2.63
4.11
2.23
4.71
4.71
9.26
2.63
3.05
2.34
3.55
6.61
2.63
2.63
4.71
8.59
5.33
3.55
7.26
2.63
5.96
4.71
9.93
3.55
4.12
3.55
7.26
3.05
4.12
2.23
3.62
7.68
3.42
6.47
3.94
3.94
3.62
6.47
3.42
10.24
3.94
4.34
3.42
4.34
3.62
3.94
3.94
6.47
4.81
4.34
3.42
3.42
3.62
3.62
9.59
5.33
10.9
3.62
3.42
3.62
7.68
10.24
3.62
4.34
3.35
7.68
3.94
3.94
4.34
3.62
3.35
10.9
3.94
3.35
7.07
5.33
7.68
3.62
4.81
10.24
4.81
3.62
7.68
7.68
11.56
4.34
5.33
3.94
4.34
6.47
7.68
3.35
5.33
3.42
3.62
3.94
11.56
3.94
3.62
9.59
3.94
3.94
3.62
3.42
5.33
6.47
5.33
7.68
3.94
5.33
4.34
3.

2.98
6.32
1.78
1.78
4.3
8.37
4.97
2.98
7.0
1.78
5.64
4.3
9.74
2.98
3.63
2.98
7.0
2.36
3.63
1.12
5.21
8.55
5.07
7.47
5.44
5.44
5.21
7.47
5.07
10.9
5.44
5.74
5.07
5.74
5.21
5.44
5.44
7.47
6.1
5.74
5.07
5.07
5.21
5.21
10.29
6.51
11.52
5.21
5.07
5.21
8.54
10.91
5.21
5.74
5.02
8.55
5.44
5.44
5.73
5.21
5.02
11.52
5.44
5.02
8.0
6.52
8.54
5.21
6.1
10.9
6.1
5.21
8.55
8.55
12.15
5.73
6.52
5.44
5.74
7.47
8.55
5.02
6.51
5.07
5.21
5.44
12.15
5.44
5.21
10.3
5.44
5.44
5.21
5.07
6.52
7.47
6.52
8.55
5.44
6.52
5.74
5.21
5.21
7.47
5.21
6.1
6.51
5.74
5.21
5.44
9.12
5.02
7.47
5.74
6.52
9.7
6.1
5.21
5.21
8.0
5.44
5.21
5.44
5.21
5.74
5.07
9.11
9.12
5.21
7.47
5.07
6.1
5.44
5.21
5.44
5.74
5.74
6.1
10.29
5.07
5.74
6.52
5.44
6.1
5.21
5.07
5.21
5.21
6.1
5.02
6.52
6.52
10.29
5.21
5.44
5.07
5.74
8.0
5.21
5.21
6.52
9.7
6.98
5.74
8.54
5.21
7.47
6.52
10.9
5.73
6.1
5.74
8.54
5.44
6.1
5.02
4.15
7.94
3.97
6.77
4.42
4.42
4.15
6.77
3.97
10.44
4.42
4.79
3.97
4.79
4.14
4.42
4.42
6.77
5.22
4.79
3.97
3.97
4.14
4.15
9.8
5.7
11.

6.1
10.9
6.1
5.21
8.55
8.55
12.15
5.73
6.52
5.44
5.74
7.47
8.55
5.02
6.51
5.07
5.21
5.44
12.15
5.44
5.21
10.3
5.44
5.44
5.21
5.07
6.52
7.47
6.52
8.55
5.44
6.52
5.74
5.21
5.21
7.47
5.21
6.1
6.51
5.74
5.21
5.44
9.12
5.02
7.47
5.74
6.52
9.7
6.1
5.21
5.21
8.0
5.44
5.21
5.44
5.21
5.74
5.07
9.11
9.12
5.21
7.47
5.07
6.1
5.44
5.21
5.44
5.74
5.74
6.1
10.29
5.07
5.74
6.52
5.44
6.1
5.21
5.07
5.21
5.21
6.1
5.02
6.52
6.52
10.29
5.21
5.44
5.07
5.74
8.0
5.21
5.21
6.52
9.7
6.98
5.74
8.54
5.21
7.47
6.52
10.9
5.73
6.1
5.74
8.54
5.44
6.1
5.02
1.78
7.0
1.31
5.64
2.36
2.36
1.78
5.64
1.31
9.74
2.36
2.98
1.31
2.98
1.78
2.36
2.36
5.64
3.63
2.98
1.31
1.31
1.78
1.78
9.06
4.3
10.43
1.78
1.31
1.78
7.0
9.74
1.78
2.98
1.12
7.0
2.36
2.36
2.98
1.78
1.12
10.43
2.36
1.12
6.32
4.3
7.0
1.78
3.63
9.74
3.63
1.78
7.0
7.0
11.12
2.98
4.3
2.36
2.98
5.64
7.0
1.12
4.3
1.31
1.78
2.36
11.12
2.36
1.78
9.06
2.36
2.36
1.78
1.31
4.3
5.64
4.3
7.0
2.36
4.3
2.98
1.78
1.78
5.64
1.78
3.63
4.3
2.98
1.78
2.36
7.69
1.12
5.64
2.98
4.3
8.37
3.6

2.98
1.78
2.36
7.69
1.12
5.64
2.98
4.3
8.37
3.63
1.78
1.78
6.32
2.36
1.78
2.36
1.78
2.98
1.31
7.69
7.69
1.78
5.64
1.31
3.63
2.36
1.78
2.36
2.98
2.98
3.63
9.06
1.31
2.98
4.3
2.36
3.63
1.78
1.31
1.78
1.78
3.63
1.12
4.3
4.3
9.06
1.78
2.36
1.31
2.98
6.32
1.78
1.78
4.3
8.37
4.97
2.98
7.0
1.78
5.64
4.3
9.74
2.98
3.63
2.98
7.0
2.36
3.63
1.12
3.12
7.46
2.88
6.19
3.48
3.48
3.12
6.2
2.88
10.07
3.48
3.93
2.88
3.93
3.11
3.48
3.48
6.19
4.44
3.93
2.88
2.88
3.11
3.12
9.41
5.0
10.74
3.12
2.88
3.12
7.45
10.07
3.12
3.93
2.79
7.46
3.48
3.48
3.93
3.11
2.79
10.74
3.48
2.79
6.82
5.0
7.45
3.11
4.44
10.07
4.44
3.11
7.46
7.46
11.41
3.93
5.0
3.48
3.93
6.2
7.46
2.79
5.0
2.88
3.11
3.48
11.41
3.48
3.11
9.41
3.48
3.48
3.12
2.88
5.0
6.2
5.0
7.46
3.48
5.0
3.93
3.12
3.12
6.2
3.11
4.44
5.0
3.93
3.12
3.48
8.1
2.79
6.2
3.93
5.0
8.75
4.44
3.11
3.12
6.82
3.48
3.12
3.48
3.12
3.93
2.88
8.1
8.1
3.12
6.2
2.88
4.44
3.48
3.11
3.48
3.93
3.93
4.44
9.41
2.88
3.93
5.0
3.48
4.44
3.12
2.88
3.11
3.11
4.44
2.79
5.0
5.0
9.41
3.11
3.48
2.

3.93
6.82
3.12
3.12
5.0
8.75
5.59
3.93
7.45
3.12
6.19
5.0
10.07
3.93
4.44
3.93
7.45
3.48
4.44
2.79
1.78
7.0
1.31
5.64
2.36
2.36
1.78
5.64
1.31
9.74
2.36
2.98
1.31
2.98
1.78
2.36
2.36
5.64
3.63
2.98
1.31
1.31
1.78
1.78
9.06
4.3
10.43
1.78
1.31
1.78
7.0
9.74
1.78
2.98
1.12
7.0
2.36
2.36
2.98
1.78
1.12
10.43
2.36
1.12
6.32
4.3
7.0
1.78
3.63
9.74
3.63
1.78
7.0
7.0
11.12
2.98
4.3
2.36
2.98
5.64
7.0
1.12
4.3
1.31
1.78
2.36
11.12
2.36
1.78
9.06
2.36
2.36
1.78
1.31
4.3
5.64
4.3
7.0
2.36
4.3
2.98
1.78
1.78
5.64
1.78
3.63
4.3
2.98
1.78
2.36
7.69
1.12
5.64
2.98
4.3
8.37
3.63
1.78
1.78
6.32
2.36
1.78
2.36
1.78
2.98
1.31
7.69
7.69
1.78
5.64
1.31
3.63
2.36
1.78
2.36
2.98
2.98
3.63
9.06
1.31
2.98
4.3
2.36
3.63
1.78
1.31
1.78
1.78
3.63
1.12
4.3
4.3
9.06
1.78
2.36
1.31
2.98
6.32
1.78
1.78
4.3
8.37
4.97
2.98
7.0
1.78
5.64
4.3
9.74
2.98
3.63
2.98
7.0
2.36
3.63
1.12
1.78
7.0
1.31
5.64
2.36
2.36
1.78
5.64
1.31
9.74
2.36
2.98
1.31
2.98
1.78
2.36
2.36
5.64
3.63
2.98
1.31
1.31
1.78
1.78
9.06
4.3
10.43
1.78
1.

4.3
1.31
1.78
2.36
11.12
2.36
1.78
9.06
2.36
2.36
1.78
1.31
4.3
5.64
4.3
7.0
2.36
4.3
2.98
1.78
1.78
5.64
1.78
3.63
4.3
2.98
1.78
2.36
7.69
1.12
5.64
2.98
4.3
8.37
3.63
1.78
1.78
6.32
2.36
1.78
2.36
1.78
2.98
1.31
7.69
7.69
1.78
5.64
1.31
3.63
2.36
1.78
2.36
2.98
2.98
3.63
9.06
1.31
2.98
4.3
2.36
3.63
1.78
1.31
1.78
1.78
3.63
1.12
4.3
4.3
9.06
1.78
2.36
1.31
2.98
6.32
1.78
1.78
4.3
8.37
4.97
2.98
7.0
1.78
5.64
4.3
9.74
2.98
3.63
2.98
7.0
2.36
3.63
1.12
6.3
9.25
6.18
8.26
6.48
6.48
6.3
8.27
6.18
11.46
6.48
6.74
6.18
6.74
6.29
6.48
6.48
8.26
7.05
6.74
6.18
6.18
6.29
6.3
10.88
7.41
12.05
6.3
6.18
6.3
9.24
11.47
6.3
6.74
6.14
9.25
6.48
6.48
6.73
6.29
6.14
12.05
6.48
6.14
8.74
7.41
9.24
6.29
7.05
11.46
7.05
6.29
9.25
9.25
12.65
6.73
7.41
6.48
6.74
8.27
9.25
6.14
7.41
6.18
6.29
6.48
12.65
6.48
6.29
10.89
6.48
6.48
6.3
6.18
7.41
8.27
7.41
9.25
6.48
7.41
6.74
6.3
6.3
8.27
6.29
7.05
7.41
6.74
6.3
6.48
9.78
6.14
8.27
6.74
7.41
10.32
7.05
6.29
6.3
8.74
6.48
6.3
6.48
6.3
6.74
6.18
9.77
9.78
6.3
8.

In [50]:
#Tells you the data above is within 5 miles or not by outputting: True or False

Locations_less_than_5_miles=[]
for i in dtf['Longitude']:
    for x in dtf['Latitude']:
        radiusEarth= 3961 #miles. If you want km, put in 6373 km in instead
        lat1 = math.radians(36.14)
        lon1 = math.radians(-115.19)
        lat2 = math.radians(x)  #plug and chug
        lon2 = math.radians(i)  #plug and chug
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = round(radiusEarth * c,2) #in miles
        if distance < 5:  #5 mile threshold
            Locations_less_than_5_miles.append(True)
        else:
            Locations_less_than_5_miles.append(False)
Locations_less_than_5_miles[:]

[True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,


In [51]:
#Combine the data and the corresponding

ID_Locations_less_than_5_miles=pd.Series(Locations_less_than_5_miles)
dtf["Within_5_miles"] = ID_Locations_less_than_5_miles[:] #Index the true and false for within 5 miles

#This is a random string generator for the # of evaluations
dtf["% of People Infected"] = np.random.choice(["%80 > Infected","%60 > Infected","%40 > Infected","%20 > Infected","%20 < Infected"], size=len(dtf), p=[0.2,0.2,0.05,0.40,0.15])

#dtf["# of People Processed"] = np.random.choice(["People using app:Greater than 66","People using app:Between 33-66","People using app:Lower than 33"], size=len(dtf), p=[0.4,0.45,0.15])

# The fraction of people infected from the # of evaluations
dtf["# of People Processed"] = np.random.randint(low=0, high=100, size=len(dtf))
#dtf["Infected%"] = np.random.randint(low=0, high=66, size=len(dtf))

#Toggle to play with # of data points will output
#dtf.head(10)

In [52]:
#Display only data points within 5 miles only!
dtf[dtf.Within_5_miles==True]

,id,City,Street Address,Longitude,Latitude,Within_5_miles,% of People Infected,# of People Processed
0,0,Las Vegas,4507 Flamingo Rd,-115.20,36.12,True,%20 < Infected,77
2,2,Las Vegas,"3200 LAS VEGAS BLVD. S., STE 1795",-115.17,36.13,True,%20 > Infected,67
4,4,Las Vegas,3730 LAS VEGAS BLVD S,-115.18,36.11,True,%20 < Infected,7
5,5,Las Vegas,129 East Fremont Street,-115.14,36.17,True,%60 > Infected,51
6,6,Las Vegas,3475 Las Vegas Blvd S,-115.17,36.12,True,%40 > Infected,37
...,...,...,...,...,...,...,...,...
150,150,Las Vegas,3900 Las Vegas Blvd,-115.18,36.09,True,%80 > Infected,10
151,151,Las Vegas,3850 Las Vegas Blvd So,-115.18,36.10,True,%20 > Infected,18
153,153,Las Vegas,"9701 W. Flamingo Road, #1",-115.30,36.11,True,%20 > Infected,26
154,154,Las Vegas,3950 Las Vegas Blvd So,-115.17,36.09,True,%20 > Infected,70


In [53]:
#Initialize and center the map to be simulated in Las Vegas

city = "Las Vegas"
## get location
locator = geopy.geocoders.Nominatim(user_agent="MyCoder")
#location = locator.geocode(city)
#print(location)
## keep latitude and longitude only
location = [36.14, -115.19]  #3301 W. Sahara Ave
print("[lat, long]:", location)

[lat, long]: [36.14, -115.19]


In [54]:
x, y = "Latitude", "Longitude"
color = "% of People Infected"
size = "# of People Processed"
#popup = "Cost" & "Street Address"

popup = "Street Address"
data = dtf[dtf.Within_5_miles==True]

## create color column
lst_colors=["green","lightgreen","pink","orange","red"]

#lst_colors=["red","pink","orange","darkblue","black"]

lst_elements = sorted(list(dtf[color].unique()))
data["color"] = data[color].apply(lambda x: 
                lst_colors[lst_elements.index(x)])
## create size column (scaled)
scaler = preprocessing.MinMaxScaler(feature_range=(3,15))
data["size"] = scaler.fit_transform(
               data[size].values.reshape(-1,1)).reshape(-1)

## initialize the map with the starting location
map_ = folium.Map(location=location, tiles="cartodbpositron",
                  zoom_start=11)
## add points
data.apply(lambda row: folium.CircleMarker(
           location=[row[x],row[y]], popup=row[popup],
           color=row["color"], fill=True,
           radius=row["size"]).add_to(map_), axis=1)
## add html legend
legend_html = """<div style="position:fixed; bottom:10px; left:10px; border:2px solid black; z-index:9999; font-size:14px;">&nbsp;<b>"""+color+""":</b><br>"""
for i in lst_elements:
     legend_html = legend_html+"""&nbsp;<i class="fa fa-circle 
     fa-1x" style="color:"""+lst_colors[lst_elements.index(i)]+"""">
     </i>&nbsp;"""+str(i)+"""<br>"""
legend_html = legend_html+"""</div>"""
map_.get_root().html.add_child(folium.Element(legend_html))

## plot the map
map_

<ipython-input-54-1271a957e97f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["color"] = data[color].apply(lambda x:
<ipython-input-54-1271a957e97f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["size"] = scaler.fit_transform(
